# Chp13 Best practices for the real world

* https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html

In [1]:
!pip install keras-tuner -q

In [2]:
def HR():
    print('-' * 80)

# This results in an error in listing 12.31!
def set_mixed_precision():
    import tensorflow as tf
    from tensorflow import keras

    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        print("GPU mode - switch to mixed precision.")
        print("Every layer will use a 16-bit compute dtype and float32 variable dtype by default.")
        keras.mixed_precision.set_global_policy("mixed_float16")

    HR()
    print("global policy:", tf.keras.mixed_precision.global_policy())

set_mixed_precision()

GPU mode - switch to mixed precision.
Every layer will use a 16-bit compute dtype and float32 variable dtype by default.
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5
--------------------------------------------------------------------------------
global policy: <Policy "mixed_float16">


In [3]:
# Listing 13.1 A KerasTuner model-building function, p.472

from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    
    return model

In [4]:
# Listing 13.2 A KerasTuner HyperModel, p.473

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        
        return model

hypermodel = SimpleMLP(num_classes=10)

In [5]:
def get_best_epoch(hp, data):
    
    x_train = data['x_train']
    y_train = data['y_train']
    x_val = data['x_val']
    y_val = data['y_val']

    model = build_model(hp)
    
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", 
            mode="min", 
            patience=2,
            #patience=10,
        )
    ]

    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        # epochs=100,
        epochs=2,
        batch_size=128,
        callbacks=callbacks,
        verbose=2,
    )
    
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print()
    print(f"Best epoch: {best_epoch}")
    
    return best_epoch

In [6]:
def get_best_trained_model(hp, data):

    x_train_full = data['x_train_full']
    y_train_full = data['y_train_full']

    # GB added this line here
    model = build_model(hp)

    best_epoch = get_best_epoch(hp, data)

    # HR()
    # print(f"best_epoch * 1.2: {int(best_epoch * 1.2)}")

    model.fit(
        x_train_full, 
        y_train_full,
        batch_size=128, 
        epochs=int(best_epoch * 1.2),
        verbose=2
    )
    return model

In [14]:
def create_data():

    # Load the data
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Normalize pixel values between 0 and 1
    x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
    x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255

    x_train_full = x_train[:]
    y_train_full = y_train[:]

    num_val_samples = 10_000
    x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
    y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]

    data = {
        'x_train_full': x_train_full,
        'y_train_full': y_train_full,
        'x_train': x_train,
        'y_train': y_train,
        'x_test': x_test,
        'y_test': y_test,
        'x_val': x_val,
        'y_val': y_val
    }

    return data

data = create_data()
print(data.keys())

dict_keys(['x_train_full', 'y_train_full', 'x_train', 'y_train', 'x_test', 'y_test', 'x_val', 'y_val'])


In [25]:
def tuner_model(data, tuner):

    x_train = data['x_train']
    y_train = data['y_train']
    x_val = data['x_val']
    y_val = data['y_val']
    x_test = data['x_test']
    y_test = data['y_test']


    ################

    # display an overview of the search space via search_space_summary():
    tuner.search_space_summary()

    #############

    # Create a callback to stop training early after reaching a certain value for the validation loss
    callbacks = [
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
    ]


    # Run the hyperparameter search
    # This method has the same signature as keras.Model.fit
    tuner.search(
        x_train, y_train,
        batch_size=128,
        epochs=3,
        # epochs=100,
        validation_data=(x_val, y_val),
        callbacks=callbacks,
        verbose=2,
    )

    ################

    # Listing 13.3 Querying the best hyperparameter configurations, p.476
    top_n = 4

    # # Get the optimal hyperparameters
    best_hps = tuner.get_best_hyperparameters(top_n)

    ################

    best_models = []

    for hp in best_hps:
        model = get_best_trained_model(hp, data)
        model.evaluate(x_test, y_test)
        best_models.append(model)
        HR()


Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available:

    BayesianOptimization
    Hyperband
    RandomSearch
    Sklearn


In [31]:
import sklearn.model_selection
import keras_tuner as kt

# BayesianOptimization Tuner
# https://keras.io/api/keras_tuner/tuners/bayesian/

tuner1 = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    #max_trials=100,
    max_trials=10,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

    # hypermodel,
    # objective,
    # max_trials,
    # num_initial_points=2,
    # alpha=0.0001,
    # beta=2.6,
    # seed=None,
    # hyperparameters=None,
    # tune_new_entries=True,
    # allow_new_entries=True,
    # **kwargs

tuner_model(data, tuner1)

Trial 10 Complete [00h 00m 11s]
val_accuracy: 0.9555500149726868

Best val_accuracy So Far: 0.9599000215530396
Total elapsed time: 00h 01m 55s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
391/391 - 2s - loss: 0.4215 - accuracy: 0.8867 - val_loss: 0.2319 - val_accuracy: 0.9373
Epoch 2/2
391/391 - 1s - loss: 0.2186 - accuracy: 0.9370 - val_loss: 0.1786 - val_accuracy: 0.9522

Best epoch: 2
Epoch 1/2
469/469 - 2s - loss: 0.3964 - accuracy: 0.8929
Epoch 2/2
469/469 - 1s - loss: 0.2075 - accuracy: 0.9411
313/313 [==============================] - 1s 2ms/step - loss: 0.1839 - accuracy: 0.9462
--------------------------------------------------------------------------------
Epoch 1/2
391/391 - 2s - loss: 0.4261 - accuracy: 0.8853 - val_loss: 0.2318 - val_accuracy: 0.9366
Epoch 2/2
391/391 - 1s - loss: 0.2178 - accuracy: 0.9380 - val_loss: 0.1845 - val_accuracy: 0.9489

Best epoch: 2
Epoch 1/2
469/469 - 2s - loss: 0.3811 - accuracy: 0.8975
Epoch 2/2
469/469 - 1s - loss: 0.1907 - accuracy: 0.

In [32]:
import sklearn.model_selection
import keras_tuner as kt

# https://keras.io/api/keras_tuner/tuners/hyperband/
# Hyperband Tuner

tuner2 = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    # max_epochs=10,
    max_epochs=5,
    factor=3,
    directory='mnist_kt_test',
    project_name='intro_to_kt'
)

    # hypermodel,
    # objective,
    # max_epochs,
    # factor=3,
    # hyperband_iterations=1,
    # seed=None,
    # hyperparameters=None,
    # tune_new_entries=True,
    # allow_new_entries=True,
    # **kwargs

tuner_model(data, tuner2)

INFO:tensorflow:Reloading Oracle from existing project mnist_kt_test/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from mnist_kt_test/intro_to_kt/tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
391/391 - 2s - loss: 0.4627 - accuracy: 0.8747 - val_loss: 0.2421 - val_accuracy: 0.9321
Epoch 2/2
391/391 - 1s - loss: 0.2200 - accuracy: 0.9376 - val_loss: 0.1786 - val_accuracy: 0.9511

Best epoch: 2
Epoch 1/2
469/469 - 2s - loss: 0.4095 - accuracy: 0.8886
Epoch 2/2
469/469 - 1s - loss: 0.2010 - accuracy: 0.9444
313/313 [==============================] - 1s 2ms/step - loss: 0.1744 - accuracy: 0.9484
--------------------------------------------------------------------------------
Epoch 1/2
391/391 - 2s - l

In [33]:
import sklearn.model_selection
import keras_tuner as kt

# https://keras.io/api/keras_tuner/tuners/random/
# RandomSearch Tuner

tuner3 = kt.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    directory='mnist_kt_test',
)

    # hypermodel,
    # objective,
    # max_trials,
    # seed=None,
    # hyperparameters=None,
    # tune_new_entries=True,
    # allow_new_entries=True,
    # **kwargs

tuner_model(data, tuner3)

INFO:tensorflow:Reloading Oracle from existing project mnist_kt_test/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from mnist_kt_test/untitled_project/tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
391/391 - 2s - loss: 0.5007 - accuracy: 0.8706 - val_loss: 0.2856 - val_accuracy: 0.9198
Epoch 2/2
391/391 - 1s - loss: 0.2697 - accuracy: 0.9240 - val_loss: 0.2222 - val_accuracy: 0.9376

Best epoch: 2
Epoch 1/2
469/469 - 2s - loss: 0.4701 - accuracy: 0.8743
Epoch 2/2
469/469 - 1s - loss: 0.2452 - accuracy: 0.9295
313/313 [==============================] - 1s 2ms/step - loss: 0.2170 - accuracy: 0.9361
--------------------------------------------------------------------------------
Epoch 1/2
391/39

In [54]:
import sklearn.model_selection
import keras_tuner as kt

# https://keras.io/api/keras_tuner/tuners/base_tuner/
# The base Tuner class

tuner0 = kt.Tuner(
    oracle=kt.oracles.BayesianOptimizationOracle(
        objective=kt.Objective('score', 'max'),
        max_trials=10
    ),
    hypermodel=build_model, #hypermodel,
    max_model_size=None,
    optimizer=None,
    loss=None,
    metrics=None,
    distribution_strategy=None,
    directory="mnist_kt_test",
    project_name=None,
    logger=None,
    tuner_id=None,
    overwrite=False,
)

    # oracle,
    # hypermodel,
    # max_model_size=None,
    # optimizer=None,
    # loss=None,
    # metrics=None,
    # distribution_strategy=None,
    # directory=None,
    # project_name=None,
    # logger=None,
    # tuner_id=None,
    # overwrite=False,

tuner_model(data, tuner0)

INFO:tensorflow:Reloading Oracle from existing project mnist_kt_test/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from mnist_kt_test/untitled_project/tuner0.json
Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
391/391 - 2s - loss: 0.4202 - accuracy: 0.8870 - val_loss: 0.2247 - val_accuracy: 0.9388
Epoch 2/2
391/391 - 1s - loss: 0.2151 - accuracy: 0.9379 - val_loss: 0.1751 - val_accuracy: 0.9543

Best epoch: 2
Epoch 1/2
469/469 - 2s - loss: 0.3783 - accuracy: 0.8972
Epoch 2/2
469/469 - 1s - loss: 0.1919 - accuracy: 0.9453
313/313 [==============================] - 1s 2ms/step - loss: 0.1637 - accuracy: 0.9532
--------------------------------------------------------------------------------
Epoch 1/2
391/39

In [75]:
# #import sklearn.model_selection
# import keras_tuner as kt
# from sklearn import model_selection
# from sklearn import metrics
# from sklearn.pipeline import Pipeline


# # Tuning Scikit-learn Models


# # https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html
# # Despite its name, Keras Tuner can be used to tune a wide variety of 
# # machine learning models. In addition to built-in Tuners for Keras models, 
# # Keras Tuner provides a built-in Tuner that works with Scikit-learn models. 
 

# # https://keras.io/api/keras_tuner/tuners/sklearn/
# # Sklearn Tuner

# tuner4 = kt.tuners.SklearnTuner(
#     oracle=kt.oracles.BayesianOptimizationOracle(
#         objective=kt.Objective('score', 'loss'),
#         max_trials=10
#     ),
#     hypermodel=build_model,
#     # scoring=None, 
#     # metrics=None, 
#     scoring=metrics.make_scorer(metrics.accuracy_score),
#     cv=model_selection.StratifiedKFold(5),
#     directory='mnist_kt_test',
#     project_name='my_project',
# )

# X_train = data['x_train']
# y_train = data['y_train']

# tuner4.search(X_train, y_train)

# best_model = tuner4.get_best_models(num_models=1)[0]

#     # hypermodel,
#     # objective,
#     # max_epochs,
#     # factor=3,
#     # hyperband_iterations=1,
#     # seed=None,
#     # hyperparameters=None,
#     # tune_new_entries=True,
#     # allow_new_entries=True,
#     # **kwargs

